In [12]:
from datetime import datetime

import time
import os
import sys
import itertools
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.time import Time
from astropy.modeling import models, fitting
import astropy.units as u

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams.update({'font.size': 8})

import seaborn as sns

np.random.seed(42)

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
def load_wfs_data(files=[f"../raw_data/2018_2019_wfs_loose.csv"]):
    # load data and make some subsets for each wfs for inspection later
    dfs = []
    for f in files:
        dfs.append(pd.read_csv(f))
    data = pd.concat(dfs)
    data['ut'] = pd.to_datetime(data.ut)
    data['az'][data['az'] < 0.] += 360.

    f9 = data[(data['wfs'] == 'newf9') | (data['wfs'] == 'oldf9')]
    f5 = data[data['wfs'] == 'f5']
    mmirs = data[data['wfs'] == 'mmirs']
    bino = data[data['wfs'] == 'binospec']

    # wrangle the times to add colums for mjd to look for trends over time and hour to look for nightly trends
    raw_times = data['time']
    times = Time(raw_times.values.tolist(), format='isot', scale='utc')
    mjd = times.mjd
    data['mjd'] = mjd.tolist()
    data['hour'] = data['ut'].dt.hour

    e_series = pd.read_csv("../halcoll/data/halcoll_temps.csv")

    fixed = data.drop(columns=['ut']).set_index(pd.DatetimeIndex(data['time'], name='ut'))

    e_series = e_series.set_index(pd.DatetimeIndex(e_series['ts'], name='ut').tz_localize('MST').tz_convert(None)).drop(columns=['ts'])

    merged = pd.merge_asof(fixed.sort_index(), e_series, on='ut')

    # trim out columns not relevant to training
    dropped_cols = [
        'ut', 'time', 'airmass', 'cc_x_err', 'cc_y_err', 'chamt', 'osst',
        'outt', 'exptime', 'file', 'focerr', 'fwhm', 'raw_seeing', 'residual_rms',
        'seeing', 'wavefront_rms', 'xcen', 'ycen', 'comaerr'
    ]
    trimmed = merged.drop(columns=dropped_cols)
    trimmed = trimmed.dropna()
    trimmed['m1temp'] = trimmed[trimmed.columns[trimmed.columns.str.contains(pat='plate')]].mean(axis=1)
    trimmed['osstemp'] = trimmed[trimmed.columns[trimmed.columns.str.contains(pat='temptrax')]].mean(axis=1)
    trimmed['meantemp'] = (trimmed['m1temp'] + trimmed['osstemp']) / 2.
    
    return trimmed

def norm_dataset(x, s):
    return (x - s['mean']) / s['std']

In [4]:
labels = ['focus', 'tiltx', 'tilty', 'transx', 'transy']
trimmed = load_wfs_data(files=["../raw_data/2018_2019_wfs_loose.csv"])

# split data set up into three training sets: f9, mmirs, and f5/bino. these sets correspond to the three optical configurations
# we use: f/9 and no corrector, f/5 with no corrector (mmirs), and f/5 with spectroscopic corrector (f5/bino).
datasets = {}
wfs = trimmed.pop('wfs')
datasets['f9'] = trimmed[(wfs == 'newf9') | (wfs == 'oldf9')]
datasets['mmirs'] = trimmed[wfs == 'mmirs']
datasets['f5'] = trimmed[(wfs == 'f5') | (wfs == 'binospec')]

# the large range in offsets is messing up the training. so normalize the hexapod coords to their means
means = {}
train_datasets = {}
test_datasets = {}
train_labels = {}
test_labels = {}
train_stats = {}
normed_train_data = {}
normed_test_data = {}
for w in datasets:
    means[w] = {}
    for l in labels:
        means[w][l] = datasets[w][l].mean()
        # datasets[w][l] -= means[w][l]

    train_datasets[w] = datasets[w].sample(frac=0.75, random_state=0)
    test_datasets[w] = datasets[w].drop(train_datasets[w].index)
    train_stats[w] = train_datasets[w].describe()
    train_stats[w] = train_stats[w].drop(columns=labels)
    train_stats[w] = train_stats[w].transpose()
    
    train_labels[w] = {}
    test_labels[w] = {}
    for l in labels:
        train_labels[w][l] = train_datasets[w].pop(l)
        test_labels[w][l] = test_datasets[w].pop(l)
        
    normed_train_data[w] = norm_dataset(train_datasets[w], train_stats[w])
    normed_test_data[w] = norm_dataset(test_datasets[w], train_stats[w])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
datasets['f9'].columns

Index(['az', 'el', 'focus', 'tiltx', 'tilty', 'transx', 'transy', 'mjd',
       'hour', 'cell_e_series_backplate_C', 'cell_e_series_chamber_ambient_C',
       'cell_e_series_frontplate_C', 'cell_e_series_in_front_of_primary_C',
       'cell_e_series_lower_plenum_C', 'cell_e_series_midplate_C',
       'cell_e_series_outside_ambient_C', 'yankee_temperature',
       'temptrax1_probe2', 'temptrax1_probe3', 'temptrax1_probe4',
       'temptrax1_probe6', 'temptrax3_probe10', 'temptrax3_probe11',
       'temptrax3_probe12', 'm1temp', 'osstemp', 'meantemp'],
      dtype='object')

In [6]:
data = datasets['f5'].copy()

In [7]:
data.columns

Index(['az', 'el', 'focus', 'tiltx', 'tilty', 'transx', 'transy', 'mjd',
       'hour', 'cell_e_series_backplate_C', 'cell_e_series_chamber_ambient_C',
       'cell_e_series_frontplate_C', 'cell_e_series_in_front_of_primary_C',
       'cell_e_series_lower_plenum_C', 'cell_e_series_midplate_C',
       'cell_e_series_outside_ambient_C', 'yankee_temperature',
       'temptrax1_probe2', 'temptrax1_probe3', 'temptrax1_probe4',
       'temptrax1_probe6', 'temptrax3_probe10', 'temptrax3_probe11',
       'temptrax3_probe12', 'm1temp', 'osstemp', 'meantemp'],
      dtype='object')

In [8]:
fixed = {
    "c0_0": False,
    "c1_0": False,
    "c2_0": False,
    "c0_1": False,
    "c0_2": True,
    "c1_1": True
}
model = models.Polynomial2D(degree=2, fixed=fixed)
fitter = fitting.LevMarLSQFitter()

In [9]:
p = fitter(model, data['el'], data['meantemp'], data['focus'])

In [10]:
p

<Polynomial2D(2, c0_0=13780.81126851, c1_0=30.62423993, c2_0=-0.19276644, c0_1=-36.48238129, c0_2=0., c1_1=0.)>

In [15]:
el_range = np.arange(0, 90, 0.2)
t_range = np.arange(data['meantemp'].min(), data['meantemp'].max(), 0.1)
resid = data['focus'] - p(data['el'], data['meantemp'])
Xs, Ys = np.meshgrid(el_range, t_range)
fig = plt.figure()
plt.tight_layout()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data['el'], data['meantemp'], data['focus'])
ax.plot_surface(Xs, Ys, p(Xs, Ys))
ax.set_xlabel("Elevation (deg)")
ax.set_ylabel("Telescope Temp ($^{\circ}$C)")
ax.set_zlabel("Focus ($\mu$m)")
plt.savefig("f5.pdf")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plt.close('all')

In [16]:
resid.std()

167.7678167280546

In [17]:
t_diff = data['osstemp'] - data['m1temp']
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(resid, t_diff)
ax.set_xlabel("Focus Residual ($\mu$m)")
ax.set_ylabel("$T_{OSS} - T_{M1}$ ($^{\circ}C$)")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …